# Titanic Challenge

Useful libraries import

In [1]:
import pandas as pd
import matplotlib as plt
import numpy as np
import sklearn as sl
import matplotlib.pyplot as plt

## Importing training set

In [2]:
training_set = pd.read_csv('/kaggle/input/titanic/train.csv')

In [3]:
training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [4]:
training_set.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Importing testing set

In [5]:
testing_set = pd.read_csv('/kaggle/input/titanic/test.csv')

In [6]:
testing_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  418 non-null    int64  
 1   Pclass       418 non-null    int64  
 2   Name         418 non-null    object 
 3   Sex          418 non-null    object 
 4   Age          332 non-null    float64
 5   SibSp        418 non-null    int64  
 6   Parch        418 non-null    int64  
 7   Ticket       418 non-null    object 
 8   Fare         417 non-null    float64
 9   Cabin        91 non-null     object 
 10  Embarked     418 non-null    object 
dtypes: float64(2), int64(4), object(5)
memory usage: 36.0+ KB


In [7]:
testing_set.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Data selection

We assume that some of the columns are not useful to reach our goal. 
The first step is therefore selecting columns based on their (hypothetical) influence on avoiding passengers death. 

* **PassengerId** (we keep it only to have a reference);
* **Pclass** (i.e. the third class was located in bow and stern: it is likely that passengers in these positions died in different proportions);
* **Sex** (it may seem a cognitive bias, but if we study the disposition of unmarried people we can see that they were located in different parts of the ship);
* **Age** (we can assume old people were most likely to die in an emergency situation: this is a weak hypothesis);
* **Parch** (to be parent of a child might have been a reason of having more chances to get on a lifeboat);
* **Survived** (id survived or not). 

In [8]:
clean_training_set = training_set[["PassengerId", "Pclass", "Sex", "Age", "Parch", "Survived"]]

In [9]:
clean_training_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Sex          891 non-null    object 
 3   Age          714 non-null    float64
 4   Parch        891 non-null    int64  
 5   Survived     891 non-null    int64  
dtypes: float64(1), int64(4), object(1)
memory usage: 41.9+ KB


## Training set normalization

## Training set traduction (strings to numbers)

Here we convert males to value 0 and females to value 1

In [10]:
df = clean_training_set[clean_training_set['Sex'].notnull()].copy()
df['Sex'] = df['Sex'].astype(str).copy()
df["Sex"]=df.Sex.map({"male":0,"female":1})

df

,PassengerId,Pclass,Sex,Age,Parch,Survived
0,1,3,0,22.0,0,0
1,2,1,1,38.0,0,1
2,3,3,1,26.0,0,1
3,4,1,1,35.0,0,1
4,5,3,0,35.0,0,0
...,...,...,...,...,...,...
886,887,2,0,27.0,0,0
887,888,1,1,19.0,0,1
888,889,3,1,NaN,2,0
889,890,1,0,26.0,0,1


As we can see Age column has NaN values that have to be replaced with the medium value of age

In [11]:
check_nan = df['Age'].isnull().values.any()
print(check_nan)

True


In [12]:
mean_age = int(df['Age'].mean())
print("Mean age: " + str(mean_age))

Mean age: 29


In [13]:
df.loc[ pd.isna(df['Age']), 'Age'] = mean_age

In [14]:
check_nan = df['Age'].isnull().values.any()
print("Now we don't have NaN values anymore: " + str(check_nan))

Now we don't have NaN values anymore: False


In [15]:
df.isnull().sum()

PassengerId    0
Pclass         0
Sex            0
Age            0
Parch          0
Survived       0
dtype: int64

In [16]:
df

,PassengerId,Pclass,Sex,Age,Parch,Survived
0,1,3,0,22.0,0,0
1,2,1,1,38.0,0,1
2,3,3,1,26.0,0,1
3,4,1,1,35.0,0,1
4,5,3,0,35.0,0,0
...,...,...,...,...,...,...
886,887,2,0,27.0,0,0
887,888,1,1,19.0,0,1
888,889,3,1,29.0,2,0
889,890,1,0,26.0,0,1


## Now we're going to obtain a dataset made of 0 and 1